In [11]:
%load_ext autoreload
%autoreload 2

from artfinder import Crossref, load_csv, strict_filter
from artfinder.helpers import full_texts_from_urls
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
crosref = Crossref(app='artfinder', email='aapopov1@mephi.ru')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = load_csv('database/processed/barcikowski_full.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               548 non-null    string        
 1   license                 348 non-null    object        
 2   is_referenced_by_count  548 non-null    int64         
 3   link                    486 non-null    object        
 4   authors                 548 non-null    object        
 5   abstract                184 non-null    string        
 6   title                   548 non-null    string        
 7   doi                     548 non-null    string        
 8   type                    548 non-null    string        
 9   journal                 548 non-null    string        
 10  issn                    491 non-null    string        
 11  volume                  480 non-null    string        
 12  issue                   369 non-null    string    

In [4]:
df = df.sort_values(by='publication_date', ascending=False).reset_index(drop=True)
df

,publisher,license,is_referenced_by_count,link,authors,abstract,title,doi,type,journal,...,volume,issue,start_page,end_page,publication_date,references,pmcid,pmid,keywords,ablation
0,american chemical society (acs),"[{'start': {'date-parts': [[2025, 3, 28]], 'da...",0,[{'url': 'https://pubs.acs.org/doi/pdf/10.1021...,"[{'lastname': 'friedenauer', 'firstname': 'tin...",<NA>,on the mechanism and productivity of single-pu...,10.1021/acs.jpcc.4c08643,journal-article,the journal of physical chemistry c,...,<NA>,<NA>,<NA>,<NA>,2025-03-28,"[10.1021/acs.jpcc.8b04374, 10.1002/cphc.202200...",NaN,NaN,NaN,True
1,beilstein institut,"[{'start': {'date-parts': [[2025, 3, 25]], 'da...",0,[{'url': 'https://www.beilstein-journals.org/b...,"[{'lastname': 'altakroury', 'firstname': 'abde...",the potential to modify the size distribution ...,size control of nanoparticles synthesized by p...,10.3762/bjnano.16.31,journal-article,beilstein journal of nanotechnology,...,16,<NA>,407,417,2025-03-25,"[10.1016/j.bios.2015.05.073, 10.1016/j.colsurf...",NaN,NaN,NaN,True
2,elsevier bv,"[{'start': {'date-parts': [[2025, 3, 1]], 'dat...",0,[{'url': 'https://api.elsevier.com/content/art...,"[{'lastname': 'gabriel', 'firstname': 'philipp...",<NA>,surface modification of nd-fe-b feedstocks by ...,10.1016/j.jallcom.2025.179308,journal-article,journal of alloys and compounds,...,1020,<NA>,179308,<NA>,2025-03-01,"[10.1179/174328008x309690, 10.1007/bf03339106,...",NaN,NaN,NaN,True
3,beilstein institut,"[{'start': {'date-parts': [[2025, 2, 20]], 'da...",0,[{'url': 'https://www.beilstein-journals.org/b...,"[{'lastname': 'fromme', 'firstname': 'theo', '...",pulsed laser ablation in liquids (lal) is an e...,preferential enrichment and extraction of lase...,10.3762/bjnano.16.20,journal-article,beilstein journal of nanotechnology,...,16,<NA>,254,263,2025-02-20,"[10.1021/acs.chemrev.6b00468, 10.1016/j.jcis.2...",NaN,NaN,NaN,True
4,lukasiewicz research network - industrial chem...,"[{'start': {'date-parts': [[2025, 2, 19]], 'da...",0,[{'url': 'https://polimery.ichp.vot.pl/index.p...,"[{'lastname': 'paczkowska', 'firstname': 'karo...",the residual torsional strength of composite p...,post-impact residual torsional strength and ac...,10.14314/polimery.2025.1.4,journal-article,polimery,...,70,1,30,39,2025-02-19,"[10.3390/polym12061288, 10.59324/ejaset.2024.2...",NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,s. karger ag,"[{'start': {'date-parts': [[1997, 1, 1]], 'dat...",17,[{'url': 'https://www.karger.com/article/pdf/1...,"[{'lastname': 'misztal', 'firstname': 'tomasz'...",<NA>,natural and melatonin-stimulated changes in th...,10.1159/000127259,journal-article,neuroendocrinology,...,66,5,360,367,1997-01-01,None,NaN,NaN,NaN,False
544,"the nencki institute of experimental biology, ...","[{'start': {'date-parts': [[1996, 9, 30]], 'da...",3,[{'url': 'https://ane.pl/index.php/ane/article...,"[{'lastname': 'misztal', 'firstname': 'tomasz'...",secretion of all the pituitary hormones underg...,effects of melatonin infused into the iii vent...,10.55782/ane-1996-1182,journal-article,acta neurobiologiae experimentalis,...,56,3,769,778,1996-09-30,None,NaN,NaN,NaN,False
545,the kielanowski institute of animal physiology...,None,15,[{'url': 'http://www.journalssystem.com/jafs/p...,"[{'lastname': 'misztal', 'firstname': 't.', 'a...",<NA>,seasonal changes of melatonin secretion in rel...,10.22358/jafs/69583/1996,journal-article,journal of animal and feed sciences,...,5,1,35,48,1996-01-12,None,NaN,NaN,NaN,False
546,georg thieme verlag kg,None,2,[{'url': 'http://www.thieme-connect.de/product...,"[{'lastname': 'galikowski', 'firstname': 'm.',...",<NA>,video-assisted thoracoscopy for closure of a b...,10.1055/s-2007-1013771,journal-article,the thoracic and cardiovascular surgeon,...,43,01,60,61,1995-02-01,None,NaN,NaN,NaN,False


In [5]:
def get_pdf_link(link_entry: list[dict]|None) -> str|None:
    """
    Extracts the PDF link from the provided link entry."
    """

    if link_entry is None or len(link_entry) == 0:
        return None
    for entry in link_entry:
        if entry.get('content-type') == 'application/pdf':
            return entry.get('url')


In [6]:
df['full_text_link'] = df['link'].map(get_pdf_link)
full_text_links = df['full_text_link'].dropna()

In [13]:
urls = full_text_links.iloc[:20].to_list()

In [14]:
full_texts_from_urls(urls)

20 links. 10 downloaded. 8 restricted. 2 failed.
Task 10: File downloaded: download/file_10.pdf
Task 1: File downloaded: download/file_1.pdf
Task 19: File downloaded: download/file_19.pdf
Task 15: File downloaded: download/file_15.pdf
Task 14: Access denied. HTTP status: 403


filer link by .pdf

In [8]:
full_text_links.to_csv('full_text_links.csv', index=False)